# Reproducing the SCOUTER experiments
This notebook contains all code we used to reproduce the experiments done in the following paper:

>Li, L., Wang, B., Verma, M., Nakashima, Y., Kawasaki, R., & Nagahara, H. (2021). SCOUTER: Slot attention-based classifier for explainable image recognition. In Proceedings of the IEEE/CVF International Conference on Computer Vision (pp. 1046-1055).

Their code is available at: https://github.com/wbw520/scouter

## Downloading trained models
Training the models can take a long time, so there is also the option of downloading our pre-trained models. There is one model available for each experimental setup we trained.

The models that were trained during our experiments can be downloaded from [here](https://drive.google.com/file/d/1A5NdU9Be7PQUzWz30ujki-79Tw0BjrMx/view).

The `saved_model` folder should be placed in the same directory as this notebook.

## Training a model
WARNING: training can take a long time.

Whether a positive or negative SCOUTER model is trained, is controlled by the `--loss_status` argument. To train a positive SCOUTER model use `--loss_status 1`. To train a negative SCOUTER model use `--loss_status -1`.

If you do not have access to a Cuda device, use `--device cpu`.

### ImageNet

The dataset can be downloaded from: https://image-net.org/download.php

Pay attention to the fact that the dataset downloaded from the link above gives the validation set in one big folder, but the code assumes the validation set is split into one folder per label. To get the correct structure, run the following cell first:

In [ ]:
!python3 utils/sort_val.py --csv data/imagenet/LOC_val_solution.csv --img data/imagenet/images/ILSVRC/Data/CLS-LOC/val

In [ ]:
!python3 scouter/train.py --dataset ImageNet --model resnest26d --batch_size 70 --epochs 20 --num_classes 100 --use_slot true \
--use_pre false --loss_status 1 --slots_per_class 1 --power 2 --to_k_layer 3 --lambda_value 10 --vis false --channel 2048 \
--freeze_layers 0 --dataset_dir data/imagenet/images/ILSVRC/Data/CLS-LOC/ --output_dir saved_model/lambda10_plus

To resume training a model from checkpoint, use the `--resume` keyword with the path to the checkpoint.

In [ ]:
!python3 scouter/train.py --dataset ImageNet --model resnest26d --batch_size 70 --epochs 20 --num_classes 100 --use_slot true \
--use_pre false --loss_status 1 --slots_per_class 1 --power 2 --to_k_layer 3 --lambda_value 10 --vis false --channel 2048 \
--freeze_layers 0 --dataset_dir data/imagenet/images/ILSVRC/Data/CLS-LOC/ --output_dir saved_model/lambda10_plus \
--resume saved_model/lambda10_plus/ImageNet_use_slot_checkpoint.pth

### Con-text
The dataset can be downloaded from: https://staff.fnwi.uva.nl/s.karaoglu/datasetWeb/Dataset.html

In [ ]:
!python3 scouter/train.py --dataset ConText --model resnest26d --batch_size 70 --epochs 20 --num_classes 30 --use_slot true \
--use_pre false --loss_status 1 --slots_per_class 1 --power 2 --to_k_layer 3 --lambda_value 10 --vis false --channel 2048 \
--freeze_layers 0 --dataset_dir data/context/data/JPEGImages --output_dir saved_model/context_plus #--resume saved_model/context_plus/ConText_use_slot_checkpoint.pth

### CUB-200-2011
The dataset can be downloaded from: http://www.vision.caltech.edu/visipedia/CUB-200-2011.html

In [ ]:
!python3 scouter/train.py --dataset CUB200 --model resnest26d --batch_size 70 --epochs 150 --num_classes 100 --use_slot true \
--use_pre false --loss_status 1 --slots_per_class 1 --power 2 --to_k_layer 3 --lambda_value 10 --vis false --channel 2048 \
--freeze_layers 0 --dataset_dir data/cub200_2011/CUB_200_2011 --output_dir saved_model/cub200_150 #--resume saved_model/cub200_150/CUB200_use_slot_checkpoint.pth

### ACRIMA
The dataset can be downloaded from: https://figshare.com/s/c2d31f850af14c5b5232

In [ ]:
!python3 scouter/train.py --dataset ACRIMA --model resnest26d --batch_size 70 --epochs 20 --num_classes 2 --use_slot true \
--use_pre false --loss_status 1 --slots_per_class 1 --power 2 --to_k_layer 3 --lambda_value 10 --vis false --channel 2048 \
--freeze_layers 0 --dataset_dir data/acrima/Database/Images --output_dir saved_model/acrima_plus #--resume saved_model/acrima_plus2/ACRIMA_use_slot_checkpoint.pth

### Fully Connected (FC)
You can also train a model where slot attention is replaced by a simple fully connected layer by setting ``--use_slot False``. This can be done on any dataset by specifying the `--dataset` and `dataset_dir` options.

In [ ]:
!python3 scouter/train.py --dataset ImageNet --model resnest26d --batch_size 70 --epochs 20 --num_classes 100 --use_slot false \
--use_pre false --vis false --channel 2048 --freeze_layers 0 --dataset_dir data/imagenet/images/ILSVRC/Data/CLS-LOC/ \
--output_dir saved_model/imagenet_fc --device cpu #--resume saved_model/imagenet_fc/ImageNet_no_slot_checkpoint.pth


## Explanation evaluation metrics

The explanations generated by the models trained on ImageNet are evaluated using several metrics: area size, precision, Insertion Area Under Curve (IAUC), Deletion Area Under Curve (DAUC), infidelity, and sensitivity. Each of these metrics is calculated for each (image, explanation)-pair in the validation set and averaged to obtain the final score.

WARNING: calculating the metrics over the entire validations set can take a long time.

### Area size and precision
The area size metric is calculated by taking the average size of the explanations.
The precision metric is calculated by summing the pixels of the explanation that fall within the image's bounding box and dividing this by the sum of all pixels in the explanation.

To calculate these two metrics the `--area_prec` option should be given.

In [ ]:
!python3 scouter/eval.py --dataset ImageNet --model resnest26d --batch_size 70 --epochs 20 --num_classes 100 --use_slot true \
--use_pre false --loss_status 1 --slots_per_class 1 --power 2 --to_k_layer 3 --lambda_value 10 --vis false --channel 2048 \
--freeze_layers 0 --dataset_dir data/imagenet/images/ILSVRC/Data/CLS-LOC/ --output_dir saved_model/lambda10_plus/ --area_prec

### Insertion Area Under Curve (IAUC) and Deletion Area Under Curve (DAUC)
The IAUC metric measures the improvement in accuracy of the model during gradual addition of pixels based on their importance in the explanation. The DAUC metric measures the accuracy decline during gradual removal of pixels based on their importance in the explanation. 

We used the implementation from the following paper:
> Petsiuk, V., Das, A., & Saenko, K. (2018). Rise: Randomized input sampling for explanation of black-box models. arXiv preprint arXiv:1806.07421.

Their code is available at: https://github.com/eclique/RISE

To calculate these two metrics the `--auc` option should be given.

In [ ]:
!python3 scouter/eval.py --dataset ImageNet --model resnest26d --batch_size 70 --epochs 20 --num_classes 100 --use_slot true \
--use_pre false --loss_status 1 --slots_per_class 1 --power 2 --to_k_layer 3 --lambda_value 10 --vis false --channel 2048 \
--freeze_layers 0 --dataset_dir data/imagenet/images/ILSVRC/Data/CLS-LOC/ --output_dir saved_model/lambda10_plus/ --auc

### Infidelity and Sensitivity
The infidelity metric measures how much explanation can capture a model's change in prediction under influence of input pertubations. The sensitivity metric measures how much the explanation is affected by input pertubations.

We used the implementation from the following paper:
> Yeh, C. K., Hsieh, C. Y., Suggala, A., Inouye, D. I., & Ravikumar, P. K. (2019). On the (in) fidelity and sensitivity of explanations. Advances in Neural Information Processing Systems, 32, 10967-10978.

Their code is available at: https://github.com/chihkuanyeh/saliency_evaluation

To calculate these two metrics the `--saliency` option should be given.

In [ ]:
!python3 scouter/eval.py --dataset ImageNet --model resnest26d --batch_size 70 --epochs 20 --num_classes 100 --use_slot true \
--use_pre false --loss_status 1 --slots_per_class 1 --power 2 --to_k_layer 3 --lambda_value 10 --vis false --channel 2048 \
--freeze_layers 0 --dataset_dir data/imagenet/images/ILSVRC/Data/CLS-LOC/ --output_dir saved_model/lambda10_plus/ --saliency

It is also possible to calculate any combination of metrics at the same time by specifying multiple arguments.

## Confusion matrix metrics
The model trained on the ACRIMA dataset is evaluated on the validation set using various confusion matrix metrics: Area Under the Receiver Operating Characteristic Curve (ROC AUC), accuracy, precision, recall, F1, and Cohen's Kappa. 

In [ ]:
!python3 scouter/confusion_matrix_metrics.py --dataset ACRIMA --model resnest26d --batch_size 70 --epochs 20 --num_classes 2 --use_slot true \
--use_pre false --loss_status 1 --slots_per_class 1 --power 2 --to_k_layer 3 --lambda_value 10 --vis false --channel 2048 \
--freeze_layers 0 --dataset_dir data/acrima/Database/Images --output_dir saved_model/acrima_plus

## Pre-generated results
Below results we found during our experiments are shown.

### Explanation evaluation metrics

In [ ]:
with open("pre_gen_explanations.txt", 'r') as fp:
    content = fp.read()

print(content)

### Confusion matrix metrics

In [ ]:
with open("pre_gen_confusion_matrix.txt", 'r') as fp:
    content = fp.read()

print(content)